<a href="https://colab.research.google.com/github/Jatin-Khiyani/Visual-Situmlai-Reconstruction-Using-fMRI-and-Deep-Learning/blob/main/FastL2LiR/FastL2LiR_77x768_Tokens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import torch

## Importing Captions and fMRI

In [3]:
fmri = np.load('/content/drive/MyDrive/NSD_Dataset/fmri.npy').astype(np.float32)
clip_text = np.load('/content/drive/MyDrive/NSD_Dataset/Caption_Embedding_NSD_77x768.npy').astype(np.float32)
vqvae_latents = np.load('/content/drive/MyDrive/NSD_Dataset/z.npy').astype(np.float32)
# === Print shapes ===
print("fMRI shape:", fmri.shape)
print("CLIP text embeddings shape:", clip_text.shape)
print("VQ-VAE latents shape:", vqvae_latents.shape)


fMRI shape: (27750, 215, 200)
CLIP text embeddings shape: (27767, 77, 768)
VQ-VAE latents shape: (27750, 16384)


### Flattening and PreProccessing

In [8]:
# === Flatten fMRI to (N, D_fmri) ===
fmri_flat = fmri.reshape(fmri.shape[0], -1)  # shape: (27750, 43000)

# === Clip c to match samples ===
N, T, D = clip_text.shape
clip_text = clip_text.reshape(N, T * D)
clip_text = clip_text[:fmri_flat.shape[0]]  # now (27750, 7680)

# === Final sanity check ===
print("fMRI (flattened):", fmri_flat.shape)
print("CLIP text:", clip_text.shape)
print("VQ-VAE latents:", vqvae_latents.shape)

fMRI (flattened): (27750, 43000)
CLIP text: (27750, 59136)
VQ-VAE latents: (27750, 16384)


# Linear Regression L2 Reg -> Fmri to VQ-VAE


# Standard Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler().fit(fmri_flat)
X_norm = scaler_X.transform(fmri_flat)

scaler_Y = StandardScaler().fit(vqvae_latents)
Y_norm = scaler_Y.transform(vqvae_latents)




In [ ]:
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# ─── Ridge Dual Model ────────────────────────────────────────────────────────
class RidgeDual:
    def __init__(self, alpha=1.0, device=None):
        self.alpha = alpha
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.W = None
        self.b = None

    def fit(self, X_np, Y_np):
        X = torch.from_numpy(X_np).float().to(self.device)
        Y = torch.from_numpy(Y_np).float().to(self.device)

        N, D = X.shape

        K = X @ X.T  # Kernel matrix
        A = torch.linalg.solve(
            K + self.alpha * torch.eye(N, device=self.device),
            Y
        )

        self.W = X.T @ A  # (D, T)
        self.b = torch.zeros(Y.shape[1], device=self.device)  # No need if using scaled targets

    def predict(self, X_np, chunk_size=None):
        X = torch.from_numpy(X_np).float().to(self.device)
        if chunk_size is None:
            return (X @ self.W + self.b).cpu().numpy()

        outs = []
        for start in tqdm(range(0, X.shape[0], chunk_size), desc="Predicting", unit="chunk"):
            chunk = X[start:start+chunk_size]
            out = chunk @ self.W + self.b
            outs.append(out.cpu().numpy())
        return np.vstack(outs)

# ─── Train Model ─────────────────────────────────────────────────────────────
model_fz = RidgeDual(alpha=1.0)
model_fz.fit(X_norm, Y_norm)

# ─── Predict (on same normalized input) ──────────────────────────────────────
Y_pred_norm = model_fz.predict(X_norm)
Y_pred = scaler_Y.inverse_transform(Y_pred_norm)
print("Predicted shape:", Y_pred.shape)

import joblib
joblib.dump(model_fz, "/content/drive/MyDrive/NSD_Dataset/model_f_z.joblib")

Predicted shape: (27750, 7680)


['/content/drive/MyDrive/NSD_Dataset/model_f_z.joblib']

## Evaluating

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 1. MSE
mse = mean_squared_error(vqvae_latents, Y_pred)
print(f"Overall MSE: {mse:.4e}")

# 2. R² Score
r2 = r2_score(vqvae_latents, Y_pred, multioutput='variance_weighted')
print(f"Weighted R²: {r2:.4f}")

# 3. Pearson Correlation (sample-wise)
cors = []
for i in range(Y_pred.shape[0]):
    y_true = vqvae_latents[i]
    y_hat = Y_pred[i]
    if np.std(y_true) > 0 and np.std(y_hat) > 0:
        cors.append(np.corrcoef(y_true, y_hat)[0, 1])
cors = np.array(cors)
print(f"Mean sample-wise ρ: {cors.mean():.4f} ± {cors.std():.4f}")

# 4. Per-dimension R²
r2_per_dim = r2_score(vqvae_latents, Y_pred, multioutput='raw_values')
print(f"First 5 dims R²: {r2_per_dim[:5]}")
print(f"Median dim R²: {np.median(r2_per_dim):.4f}")




Overall MSE: 2.7493e-01
Weighted R²: 0.6651
Mean sample-wise ρ: 0.8310 ± 0.0414
First 5 dims R²: [0.6404062  0.65940213 0.6731678  0.6785784  0.63462466]
Median dim R²: 0.6668


# Linear Regression L2 Reg fMRI->Caption

# Standard Scaling

In [9]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler().fit(fmri_flat)
X_norm = scaler_X.transform(fmri_flat)

scaler_Y = StandardScaler().fit(clip_text)
Y_norm = scaler_Y.transform(clip_text)




In [10]:
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

# ─── Ridge Dual Model ────────────────────────────────────────────────────────
class RidgeDual:
    def __init__(self, alpha=1.0, device=None):
        self.alpha = alpha
        self.device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
        self.W = None
        self.b = None

    def fit(self, X_np, Y_np):
        X = torch.from_numpy(X_np).float().to(self.device)
        Y = torch.from_numpy(Y_np).float().to(self.device)

        N, D = X.shape

        K = X @ X.T  # Kernel matrix
        A = torch.linalg.solve(
            K + self.alpha * torch.eye(N, device=self.device),
            Y
        )

        self.W = X.T @ A  # (D, T)
        self.b = torch.zeros(Y.shape[1], device=self.device)  # No need if using scaled targets

    def predict(self, X_np, chunk_size=None):
        X = torch.from_numpy(X_np).float().to(self.device)
        if chunk_size is None:
            return (X @ self.W + self.b).cpu().numpy()

        outs = []
        for start in tqdm(range(0, X.shape[0], chunk_size), desc="Predicting", unit="chunk"):
            chunk = X[start:start+chunk_size]
            out = chunk @ self.W + self.b
            outs.append(out.cpu().numpy())
        return np.vstack(outs)

# ─── Train Model ─────────────────────────────────────────────────────────────
model_fc = RidgeDual(alpha=1.0)
model_fc.fit(X_norm, Y_norm)

# ─── Predict (on same normalized input) ──────────────────────────────────────
Y_pred_norm = model_fc.predict(X_norm)
Y_pred = scaler_Y.inverse_transform(Y_pred_norm)
print("Predicted shape:", Y_pred.shape)

import joblib
joblib.dump(model_fc, "/content/drive/MyDrive/NSD_Dataset/model_f_c.joblib")

Predicted shape: (27750, 59136)


['/content/drive/MyDrive/NSD_Dataset/model_f_c.joblib']

# Evaluating

In [11]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 1. MSE
mse = mean_squared_error(clip_text, Y_pred)
print(f"Overall MSE: {mse:.4e}")

# 2. R² Score
r2 = r2_score(clip_text, Y_pred, multioutput='variance_weighted')
print(f"Weighted R²: {r2:.4f}")

# 3. Pearson Correlation (sample-wise)
cors = []
for i in range(Y_pred.shape[0]):
    y_true = clip_text[i]
    y_hat = Y_pred[i]
    if np.std(y_true) > 0 and np.std(y_hat) > 0:
        cors.append(np.corrcoef(y_true, y_hat)[0, 1])
cors = np.array(cors)
print(f"Mean sample-wise ρ: {cors.mean():.4f} ± {cors.std():.4f}")

# 4. Per-dimension R²
r2_per_dim = r2_score(clip_text, Y_pred, multioutput='raw_values')
print(f"First 5 dims R²: {r2_per_dim[:5]}")
print(f"Median dim R²: {np.median(r2_per_dim):.4f}")




Overall MSE: 3.1046e-01
Weighted R²: 0.5993
Mean sample-wise ρ: 0.8353 ± 0.0194
First 5 dims R²: [1. 1. 1. 1. 1.]
Median dim R²: 0.6068
